In [3]:
#imports
import numpy as np

#random seed to replicate
np.random.seed(42)

In [4]:
#Nodes in the graph are defined as two arrays with the respective coordenates (x array and y array)

#Create a instance of metric TSP
def create_instance(n):
    x = np.random.rand(n)
    y = np.random.rand(n)
    
    return x, y

In [ ]:
#The follow dict allows fast access to nodes in the graph.
def init_index_map(size):
    neighMap = {}

    for i in range(size):
        neighMap.update({i : i})

    return neighMap

#The sequence array saves the current Hamiltonian Cycle
def init_sequence_array(size):
    sequence = np.arange(size, dtype=np.int32)

    return sequence


In [7]:
#Compute euclidean distance
def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1- y2)**2) 

In [8]:
#Given a sequence of nodes, compute the distance array
def compute_distance(nodes_sequence, x, y):
    siz = len(nodes_sequence - 1)
    distances = np.empty(siz)
    
    d = 0
    for i in range(siz):
        j = (i + 1) % siz
        
        x1 = x[i]
        y1 = y[i]
        
        x2 = x[j]
        y2 = y[j]
        
        distances[i] += distance(x1, y1, x2, y2)
        
    return distances

In [ ]:
def path_cost(x, y, seq):
    cost = 0
    size = len(seq) 
    for i in range(size):
        curr = i
        next = i + 1

        x1 = x[seq[curr]]
        y1 = y[seq[curr]]

        x2 = x[seq[next]]
        y2 = y[seq[next]]

        cost += compute_distance(x1, y1, x2, y2)
    
    return cost

In [ ]:
#Return the sum of all edges from the node
def remove_node(seq, nodeIdx, x, y):
    val = 0
    size = len(x)

    node = seq[nodeIdx]
    x1 = x[node]
    y1 = y[node]

    nodeIdx = (nodeIdx + 1) % size 
    neighborA = seq[nodeIdx]
    x2 = x[neighborA]
    y2 = y[neighborA]
    val += distance(x1, y1, x2, y2)

    nodeIdx = (nodeIdx - 2) % size 
    neighborB = seq[nodeIdx]
    x2 = x[neighborB]
    y2 = y[neighborB]
    val += distance(x1, y1, x2, y2)

    return val

In [ ]:
def propose_changes(seq, idx_dict, num_changes, x, y, cost, t):
    size = len(seq)
    nodes = np.random.choice(size, size=num_changes, replace=False)

    subtracted_cost = 0
    added_cost      = 0

    i = 0
    while(i < num_changes):
        nodeA = nodes[i]
        nodeB = nodes[i + 1]

        idxA = idx_dict[nodeA]
        idxB = idx_dict[nodeB]

        subtracted_cost += remove_node(seq, idxA, x, y)
        subtracted_cost += remove_node(seq, idxB, x, y)

        neighborA_idx = (idxA - 1) % size
        neighborA = seq[neighborA_idx]
        neighborB_idx = (idxA + 1) % size
        neighborB = seq[neighborB_idx]

        x1 = x[nodeB]
        y1 = x[nodeB]
        x2 = x[neighborA]
        y2 = y[neighborA]
        added_cost += distance(x1, y1, x2, y2)
        x2 = x[neighborB]
        y2 = y[neighborB]
        added_cost += distance(x1, y1, x2, y2)

        neighborA_idx = (idxB - 1) % size
        neighborA = seq[neighborA_idx]
        neighborB_idx = (idxB + 1) % size
        neighborB = seq[neighborB_idx]
        x1 = x[nodeA]
        y1 = x[nodeA]
        x2 = x[neighborA]
        y2 = y[neighborA]
        added_cost += distance(x1, y1, x2, y2)
        x2 = x[neighborB]
        y2 = y[neighborB]
        added_cost += distance(x1, y1, x2, y2)

        i += 2

    de = added_cost - subtracted_cost

    val = np.exp(-1 * de/t)
    coin = np.random.random()
    #If the new path is accepted, switch nodes
    if(coin < val):
        i = 0
        cost += de
        while(i < num_changes):
            nodeA = nodes[i]
            nodeB = nodes[i + 1]
            idxA = idx_dict[nodeA]
            idxB = idx_dict[nodeB]

            seq[idxA] = nodeB
            idx_dict[nodeB] = idxA
            seq[idxB] = nodeA
            idx_dict[nodeA] = idxB

            i += 2

In [9]:
def metropolis(x, y, seq, neighbors_dict, ti, tf, dt):
    cost = path_cost(x, y, seq)
    size = len(seq)

    t = ti
    while(t > tf):
        for _ in range(100):
            #Select the number of changes accord to a normal dist with mean = size/2 and std = size/4
            changes = np.floor(np.random.normal(size / 2, size/8))

            if(changes <= 0):
                changes = 1

            elif(changes > 0.7 * size):
                changes = np.ceil(0.7 * size)

            if(changes % 2 == 1):
                changes += 1

            propose_changes(seq, neighbors_dict, changes, x, y, cost, t)
        
        t = t * dt

SyntaxError: invalid syntax (301743243.py, line 2)